<a href="https://colab.research.google.com/github/MELAI-1/AI-FOR-PUBLIC-HEALTH/blob/main/Computer_Vision/Waste_water_treatment_plant_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Water water treatment plant detection

Author of this notebook: Steffen Knoblauch, Heidelberg University CC BY 4.0 Licence

# Workflow

*   Install YOLOv6
*   Inference with Pre-trained COCO Model
*   Creating Datasets
*   Yolov6 Annotations
*   DATA
*   Yolov6 Models
*   Custom Training
*   Evaluate YOLOv6 performance
*   Run YOLOv6 inference on test images



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%pwd

Make sure to use GPU.

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%ls

drive/  sample_data/


# Download MT-YOLOv6 repository and install requirements


In [ ]:
# Download MT-YOLOv6 repository and install requirements
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6


In [ ]:
!pip install -r requirements.txt

#Inference with Pre-trained COCO Model


Make sure to check your current working directory using the command

In [ ]:
pwd

'/content/YOLOv6'

My current working directory is /content/YOLOv6 and the drive is mounted at /content/drive. So to access the dataset, you have to start from /content/drive which is the drive.

First, lets create a weights directory inside the current YOLOv6 directory and then download a pretrained model from the YOLOv6 release. The command "mkdir" creates a new directory in the current working directory.

In [ ]:
%mkdir weights
%cd weights
!wget https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6s.pt

In [ ]:
%ls

yolov6s.pt  yolov6s.pt.1


In [ ]:
%cp -r /content/drive/MyDrive/YOLOv6_exercise/custom_dataset /content/YOLOv6/custom_dataset

In [ ]:
!pwd

/content/YOLOv6/weights


In [ ]:
%cd /content/YOLOv6
!python tools/train.py --batch 16 --conf configs/yolov6s_finetune.py --data data/dataset.yaml --fuse_ab --epochs 1 --img-size 256 --device 0

In [ ]:
#lets try with batch size 32 instead of 16
%cd /content/YOLOv6
!python tools/train.py --batch 32 --conf configs/yolov6s_finetune.py --data data/dataset.yaml --fuse_ab --epochs 10 --img-size 256 --device 0

Note that can adjusting the default
epochs to expedite results. The image size could also be adjusted to increase training speed.

Note that adjusting parameters like lower batch size can also help with low memory.

#Evaulate YOLOv6 Model Performance

YOLOv6 comes with a tools directory, one of which is for evaluation of model training. Evaluating the model's performance includes assessing the model's mean average precision (mAP), precision, and recall. For the uninitiated, a higher mAP score indicates our model is drawing the correct boxes in the right places.

Similar to the training argument, we do have parameters we can adjust for how we evaluate the model (like specifying confidence thresholds or modifying where the results are saved).


*  --data, type=str, default='./data/coco.yaml', help='dataset.yaml path')
*   -weights, type=str, default='./weights/yolov6s.pt', help='model.pt path(s)')
*  --batch-size, type=int, default=32, help='batch size')
*   --img-size, type=int, default=640, help='inference size (pixels)')
*  --conf-thres, type=float, default=0.001, help='confidence threshold')
*   --iou-thres, type=float, default=0.65, help='NMS IoU threshold')
*   --task, default='val', help='val, or speed')
*   --device, default='0', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
*   --half, default=False, action='store_true', help='whether to use fp16 infer')
*   --save_dir, type=str, default='runs/val/', help='evaluation save dir')
*   -name, type=str, default='exp', help='save evaluation results to save_dir/name')

The evaluation command we'll run is:



In [ ]:
%cd /content/YOLOv6
#make sure to enter the correct path to the best_ckpt.pt
!python tools/eval.py --data data/dataset.yaml  --weights runs/train/exp1/weights/best_ckpt.pt --img-size 256 --do_pr_metric True --plot_confusion_matrix --task val --device 0


In [ ]:
#Try out and anylze what happens when we change the iou_threshold to other values (instead of the default 0.65 - e.g. 0.35) What would you expect to change?

%cd /content/YOLOv6
#make sure to enter the correct path to the best_ckpt.pt
!python tools/eval.py --data data/dataset.yaml  --weights runs/train/exp1/weights/best_ckpt.pt --iou-thres 0.35 --img-size 256 --do_pr_metric True --plot_confusion_matrix --task val --device 0

#Inference Yolov6 on Test dataset

Similar to YOLOv6 evaluation, there's an inference script that comes with the repository's tools. We had tested this our before. This inference script allows us to pass a batch of images in a given directory on which the model will run its predictions.

In addition, this inference utility also comes with a number of arguments we can pass for things like displaying the labels on the predicted images (which is on by default), modifying the confidence level and NMS thresholds, and setting max detections.

--weights, type=str, default='weights/yolov6s.pt', help='model path(s) for inference.')

--source, type=str, default='data/images', help='the source path, e.g. image-file/dir.')

--yaml, type=str, default='data/coco.yaml', help='data yaml file.')

--img-size, type=int, default=640, help='the image-size(h,w) in inference size.')

--conf-thres, type=float, default=0.25, help='confidence threshold for inference.')

--iou-thres, type=float, default=0.45, help='NMS IoU threshold for inference.')

--max-det, type=int, default=1000, help='maximal inferences per image.')

--device, default='0', help='device to run our model i.e. 0 or 0,1,2,3 or cpu.')

--save-txt, action='store_true', help='save results to *.txt.')

--save-img, action='store_false', help='save visuallized inference results.')

--classes, nargs='+', type=int, help='filter by classes, e.g. --classes 0, or --classes 0 2 3.')

--agnostic-nms, action='store_true', help='class-agnostic NMS.')

--project, default='runs/inference', help='save inference results to project/name.')

-name, default='exp', help='save inference results to project/name.')

--hide-labels, default=False, action='store_true', help='hide labels.')

--hide-conf, default=False, action='store_true', help='hide confidences.')

--half, action='store_true', help='whether to use FP16 half-precision inference.')

We'll run this command:



In [ ]:
%cd /content/YOLOv6
!python tools/infer.py --yaml data/dataset.yaml --weights runs/train/exp1/weights/last_ckpt.pt --source custom_dataset/images/val --img-size 256 256  --project 'runs/inference/wastewater' --hide-labels --device 0
